# Overview
The goal of this project is to perform classification of written media as misinformation or not.


## Fetch Data
The [notebook](./index.ipynb) uses a [script](./download_datasets.sh) to automatically fetch the data for the project. It can be run manually to inspect the data beforehand by executing the following command:
```bash
chmod +x download_datasets.sh
sh download_datasets.sh
```

In [60]:
# fetch data using ./download_datasets.sh
!bash ./download_datasets.sh

Checking and downloading datasets...
✔ fake-news-classification.zip already exists. Skipping download.
✔ fake-and-real-news-dataset.zip already exists. Skipping download.
Extracting Kaggle datasets...
✔ liar_dataset.zip already exists. Skipping download.
✔ All datasets are ready!


## setup for NLP tools
In order to make full use of our NLP tooling we will install 
    - `punkt` for tokenization 
    - `stopwords` for removing common words
    - `en_core_web_sm` for named entity recognition

In [61]:
!python -m nltk.downloader punkt stopwords
!python -m spacy download en_core_web_sm

/Users/rob/micromamba/envs/fake-news/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /Users/rob/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/rob/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.2 MB/s eta 0:00:00MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Project

In [62]:
# project imports
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import spacy

# Data Preprocessing
The data is loaded, inspected, and assembled into a singular dataframe. The data is then preprocessed by removing stopwords, punctuation, and lemmatizing the text. The data is then split into training and testing sets.

## Loading Data
The data is sourced from three different datasets:
- [Fake & Real News]()
- [Fake News Classification]()
- [Liar Dataset]()

The data is loaded, inspected, and assembled into a singular dataframe.

### Define Data Paths

In [63]:
# define datasets paths
datasets = {
    "fake_and_real_news": {
        "fake": "datasets/fake-and-real-news/Fake.csv",
        "real": "datasets/fake-and-real-news/True.csv"
    },
    "fake_news_classification": {
        "train": "datasets/fake-news-classification/train (2).csv",
        "test": "datasets/fake-news-classification/test (1).csv",
        "evaluation": "datasets/fake-news-classification/evaluation.csv"
    },
    "liar_data": {
        "train": "datasets/liar_data/train.tsv",
        "test": "datasets/liar_data/test.tsv",
        "valid": "datasets/liar_data/valid.tsv"
    }
}



### Loading the Fake & Real News Dataset

In [64]:

# Load Fake & Real News Dataset
df_fake = pd.read_csv(datasets["fake_and_real_news"]["fake"])
df_real = pd.read_csv(datasets["fake_and_real_news"]["real"])

# Assign labels
df_fake["label"] = "fake"
df_real["label"] = "real"

# Standardize column names
df_fake.rename(columns={"title": "headline", "text": "content"}, inplace=True)
df_real.rename(columns={"title": "headline", "text": "content"}, inplace=True)

# Merge Fake & Real News
df_news = pd.concat([df_fake, df_real], ignore_index=True)

# print columns
print(df_news.columns)

Index(['headline', 'content', 'subject', 'date', 'label'], dtype='object')


In [65]:
# drop columns
df_news.drop(columns=["subject", "date", "headline"], inplace=True)

# rename content to text
df_news.rename(columns={"content": "text"}, inplace=True)

In [66]:
# print dataset info
print(df_news.head())
print(df_news["label"].value_counts())

                                                text label
0  Donald Trump just couldn t wish all Americans ...  fake
1  House Intelligence Committee Chairman Devin Nu...  fake
2  On Friday, it was revealed that former Milwauk...  fake
3  On Christmas day, Donald Trump announced that ...  fake
4  Pope Francis used his annual Christmas Day mes...  fake
label
fake    23481
real    21417
Name: count, dtype: int64


### Loading the Fake News Classification Dataset

In [67]:
# Load Fake News Classification Dataset with explicit delimiter
df_train = pd.read_csv(datasets["fake_news_classification"]["train"], delimiter=';')
df_test = pd.read_csv(datasets["fake_news_classification"]["test"], delimiter=';')
df_evaluation = pd.read_csv(datasets["fake_news_classification"]["evaluation"], delimiter=';')

# Merge train, test, and evaluation datasets
df_fake_news_class = pd.concat([df_train, df_test, df_evaluation], ignore_index=True)

# print columns
print(df_fake_news_class.columns)

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')


In [68]:
# drop columns
df_fake_news_class.drop(columns=['Unnamed: 0', 'title'], inplace=True)

In [69]:
# print dataset info
print(df_fake_news_class.head())
print(df_fake_news_class["label"].value_counts())
print(df_fake_news_class.columns)

                                                text  label
0  RAMALLAH, West Bank (Reuters) - Palestinians s...      1
1  BEIJING (Reuters) - U.S. President-elect Donal...      1
2  While the controversy over Trump s personal ta...      0
3  BEIJING (Reuters) - A trip to Beijing last wee...      1
4  There has never been a more UNCOURAGEOUS perso...      0
label
1    21924
0    18663
Name: count, dtype: int64
Index(['text', 'label'], dtype='object')


### Loading the Liar Dataset

In [70]:
# Define correct column names (LIAR dataset has 14 columns)
columns = ["id", "label", "statement", "subject", "speaker", "job", "state", "party",
           "venue", "barely-true", "false", "half-true", "mostly-true", "pants-fire", "context"]

# Load datasets with correct delimiter and column assignment
df_liar_train = pd.read_csv(datasets["liar_data"]["train"], delimiter='\t', names=columns, header=None)
df_liar_test = pd.read_csv(datasets["liar_data"]["test"], delimiter='\t', names=columns, header=None)
df_liar_valid = pd.read_csv(datasets["liar_data"]["valid"], delimiter='\t', names=columns, header=None)

# Combine datasets
df_liar = pd.concat([df_liar_train, df_liar_test, df_liar_valid], ignore_index=True)

# Map multi-class labels to binary labels
label_mapping = {
    "pants-fire": "fake",
    "false": "fake",
    "barely-true": "fake",
    "half-true": "real",
    "mostly-true": "real",
    "true": "real"
}
df_liar["label"] = df_liar["label"].map(label_mapping)

# Rename "statement" → "content" to match other datasets
df_liar.rename(columns={"statement": "content"}, inplace=True)

# Drop unnecessary columns
columns_to_drop = ["id", "speaker", "job", "state", "subject", "party", "venue", 
                   "barely-true", "false", "half-true", "mostly-true", "pants-fire", "context"]
df_liar.drop(columns=columns_to_drop, inplace=True)

# standardize column names
df_liar.rename(columns={"content": "text"}, inplace=True)

# Print dataset info
print(df_liar.head())
print(df_liar["label"].value_counts())

  label                                               text
0  fake  Says the Annies List political group supports ...
1  real  When did the decline of coal start? It started...
2  real  Hillary Clinton agrees with John McCain "by vo...
3  fake  Health care reform legislation is likely to ma...
4  real  The economic turnaround started at the end of ...
label
real    7134
fake    5657
Name: count, dtype: int64


In [71]:
print(df_liar.columns)

Index(['label', 'text'], dtype='object')


### Assemble Data
Here we assemble the data into a singular dataframe. This will involve renaming columns, dropping unnecessary columns, and adding a label column.

In [ ]:
# standardize labels
df_fake_news_class["label"] = df_fake_news_class["label"].map({1: "real", 0: "fake"})

# ensure all labels are strings
df_liar["label"] = df_liar["label"].astype(str)
df_fake_news_class["label"] = df_fake_news_class["label"].astype(str)
df_news["label"] = df_news["label"].astype(str)

# merge datasets
df = pd.concat([df_news, df_fake_news_class, df_liar], ignore_index=True)

# check standardized label distribution
print(df["label"].value_counts())

label
real    50475
fake    47801
Name: count, dtype: int64
